In [ ]:
APP_NAME = "Dataplane Frequency analysis"

In [ ]:
%load configuration.py


In [ ]:
%load setup.py


In [ ]:
%load data.py


# Creation of dataframes

The following section creates dataframes for each day of the logging phase.
Afterwards, the dataframes per date are unionized in order to generate 
dataframes which cover the information of data per week day, per work

In [ ]:
# Analysis per day of the logging phase

dfs = {}

for i in range(len(dates) - 1):
    dfs[i] = df_log.select(
        "id",
        "origin",
        "origin_id",
        "timestamp",
        "category",
        "ip",
        "username",
    ).filter(
        f"'{dates[i]}' <= timestamp AND timestamp < '{dates[i + 1]}'"
    ).filter("category='ssh' OR category='telnet'")


In [ ]:
# Analysis per weekday:
# Monday, Tuesday, ...

weekdays = range(7)
df_weekdays = {}

for day in weekdays:
    df = None
    for i in get_i_weekday(day):
        if df is None:
            df = dfs[i]
        else:
            df = df.union(dfs[i])
        
    df_weekdays[day] = df


summary_weekday = pd.DataFrame({
    "Day": [get_name_weekday(i) for i in weekdays],
    "# IPs SSH": [df_weekdays[i].select("ip").filter("category='ssh'").distinct().count() for i in weekdays],
    "# IPs Telnet": [df_weekdays[i].select("ip").filter("category='telnet'").distinct().count() for i in weekdays],
    "# IPs Total": [df_weekdays[i].select("ip").distinct().count() for i in weekdays],
    "# Requests SSH": [df_weekdays[i].select("*").filter("category='ssh'").count() for i in weekdays],
    "# Requests Telnet": [df_weekdays[i].select("*").filter("category='telnet'").count() for i in weekdays],
    "# Requests Total": [df_weekdays[i].count() for i in weekdays],
})
    
summary_weekday
    